## understand os module of python to get track your job: equivalent of hadoop webUI to monitor job

In [77]:
%%writefile mapper.py

from __future__ import print_function
import re
import sys
import os

if os.environ["mapred_task_is_map"] == "true":
    print("input_file:{}, start:{}, size:{}, job_id:{}, order_task:{}".format(
    os.environ["mapreduce_map_input_file"],
    os.environ["mapreduce_map_input_start"],
    os.environ["mapreduce_map_input_length"],
    os.environ["mapreduce_task_id"],
    os.environ["mapreduce_task_partition"]
    ))
    
for line in sys.stdin:
    pass

Overwriting mapper.py


In [78]:


%%bash



OUT_DIR="enviorn_mapper"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper.py \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null



19/09/20 11:20:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/20 11:20:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/20 11:20:24 INFO mapred.FileInputFormat: Total input files to process : 1
19/09/20 11:20:25 INFO mapreduce.JobSubmitter: number of splits:2
19/09/20 11:20:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568965302961_0019
19/09/20 11:20:26 INFO impl.YarnClientImpl: Submitted application application_1568965302961_0019
19/09/20 11:20:26 INFO mapreduce.Job: The url to track the job: http://96ec9b440a29:8088/proxy/application_1568965302961_0019/
19/09/20 11:20:26 INFO mapreduce.Job: Running job: job_1568965302961_0019
19/09/20 11:20:39 INFO mapreduce.Job: Job job_1568965302961_0019 running in uber mode : false
19/09/20 11:20:39 INFO mapreduce.Job:  map 0% reduce 0%
19/09/20 11:20:56 INFO mapreduce.Job:  map 50% reduce 0%
19/09/20 11:20:57 INFO mapreduce.Job:  map 100% reduce 0%
19/09/20 11:21:15 INFO

In [79]:
#!hdfs dfs -ls -h /data

#!hdfs dfs -text /data/test.txt

!hdfs dfs -text /user/jovyan/enviorn_mapper/part-00000


input_file:hdfs://localhost:9000/data/wiki/en_articles_part/articles-part, start:0, size:38430992, job_id:task_1568965302961_0019_m_000001, order_task:1	
input_file:hdfs://localhost:9000/data/wiki/en_articles_part/articles-part, start:38430992, size:38430993, job_id:task_1568965302961_0019_m_000000, order_task:0	


In [48]:
import os
#os.environ["HOME"]
CurDir = os.getcwd()
print(CurDir)

#os.mkdir("newDir")

import time

time.sleep(2)
os.rename('newDir', 'NewDir')

time.sleep(2)
os.rmdir('NewDir')

/home/jovyan


In [63]:
## enumerate

my_list = ['apple' ,'banana', 'grapes', 'pear']
for c, value in enumerate(my_list):
    print(c, value)


0 apple
1 
2 banana
3 grapes
4 pear


## use hdfs client to get head (10 lines) of the hdfs file  

In [95]:
%%writefile mapper.py

from __future__ import print_function
import re
import sys
import os

CHARS_IN_LINE = 9


if os.environ["mapred_task_is_map"] == "true":
    split_input_start = int(
        os.environ["mapreduce_map_input_start"]
    )//CHARS_IN_LINE
    print("split_input_start:{}".format(split_input_start))
    
for split_line_index, line in enumerate(sys.stdin):
    print("split_line_index:{}".format(split_line_index))
    line_number = split_line_index + split_input_start
    if (line_number < 10):
        print(line_number, line, sep = "\t")






Overwriting mapper.py


In [121]:


%%bash



OUT_DIR="enviorn_head"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper.py \
    -mapper "python mapper.py" \
    -input /data/test.txt \
    -output ${OUT_DIR} > /dev/null



Process is terminated.


In [94]:
!hdfs dfs -text /user/jovyan/enviorn_head/part-00000

#!hdfs dfs -text /data/test.txt

	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
0	12	Anarchism         Anarchism is often defined as a political philosophy which text: Unable to write to output stream.
1	
10	
11	
12	Hadoop is a file storage system on top of which mapreduce/yarn or spark can be used to process data.
13	
14	
15	Both programming paradigm has their own pros and cons.
16	
17	
18	Mapreduce/yarn is robust and can be scaled to any limit with cheap hardwares but is slow.
2	
3	I have created this text file for testing puposes.
4	
5	
6	I am enjoying Hadoop Mapreduce to process the big data.
7	
8	
9	Hadoop is an open source distributed file storage system.
split_input_start:0	
split_input_start:52	
split_line_index:0	
split_line_index:0	
split_line_index:1	
split_line_index:1	
split_line_index:10	
split_line_index:10	
split_line_index:11	
split_line_index:11	
split_line_index:12	
split_line_index:12	
split_line_index:13	
split_line_index:13	
split_line_index:14	
split_line_index:15	
split_line_index:16	
split_line_index:1

In [110]:
!hdfs dfs -text /data/test.txt

12	Anarchism         Anarchism is often defined as a political philosophy which text: Unable to write to output stream.





## define your own os environ 

In [103]:
%%writefile mapper.py

from __future__ import print_function
import os
import sys
import re

pattern = re.compile(os.environ["word_pattern"])


for line in sys.stdin:
    article_id, content = line.split("\t", 1)
    words = re.findall(pattern, content)
    for word in words:
        print(word, 1, sep = "\t")



Overwriting mapper.py


In [107]:


%%bash

OUT_DIR="user_enviorn"


hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null


yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar -D word_pattern="\w+\d+" \
    -files mapper.py \
    -mapper "python mapper.py" \
    -input /data/test.txt \
    -output ${OUT_DIR} > /dev/null




JAR does not exist or is not a normal file: /home/jovyan/-D


## job status from command line

In [129]:
%%writefile mapper.py

from __future__ import print_function
import os
import sys
import re



for line in sys.stdin:
    article_id, content = line.split("\t", 1)
    words = re.findall("\w+", content)
    for index, word in enumerate(words):
        print(word, 1, sep = "\t")
        print("reporter:finalstatus:processed {} words"
             .format(index+1), file=sys.stderr)



Overwriting mapper.py


In [130]:


%%bash



OUT_DIR="job_tracker"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper.py \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null



Process is terminated.


## kill a yarn application

In [138]:
## first list the running application then find application id from there and then run kill operation with that id

#!yarn application -list

!yarn application -kill application_1568965302961_0036



19/09/20 16:07:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
Killing application application_1568965302961_0036
19/09/20 16:07:15 INFO impl.YarnClientImpl: Killed application application_1568965302961_0036
